In [24]:
from mxnet.gluon import data as gdata,Trainer
import sys
import time
from mxnet import nd,autograd
import d2lzh as d2l

In [25]:
mnist_train = gdata.vision.FashionMNIST(train=True)
mnist_test = gdata.vision.FashionMNIST(train=False)
print('mnist_train len: %d,\nmnist_test len:%d' %
      (len(mnist_train), len(mnist_test)))

mnist_train len: 60000,
mnist_test len:10000


In [26]:
def get_num_workers():
    if sys.platform.startswith('win'):
        num_workers = 0
    else:
        num_workers = 4
    return num_workers

In [27]:
batch_size = 64
transformer = gdata.vision.transforms.ToTensor()
train_iter = gdata.DataLoader(mnist_train.transform_first(transformer),
                             batch_size=batch_size,shuffle=True,num_workers=get_num_workers())
test_iter = gdata.DataLoader(mnist_test.transform_first(transformer),batch_size=batch_size,
                            shuffle=False,num_workers=get_num_workers())

In [28]:
# 定义模型
input_nums = 784
output_nums = 10

def softmax(X):
    X_exp = X.exp()
    partition = X_exp.sum(axis=1,keepdims = True)
    return X_exp / partition
def net(X):
    return softmax(np.dot(X.reshape(-1,input_nums),W)+b)

In [29]:
def cross_entropy(y_hat,y):
    return -np.pick(y_hat,y).log()

def accuracy(y_hat,y):
   return (y_hat.argmax(axis=1) == y.astype('float32')).mean().asscalar()

def evaluate_accuracy(data_iter,net):
    acc_sum = 0.0
    for X,y in data_iter:
        y = y.astype('float32')
        y_hat = net(X)
        acc_sum += (y_hat.argmax(axis=1) == y).mean().asscalar()#
    return acc_sum

In [30]:
def train(net,train_iter,test_iter,num_epochs,batch_size,loss,params,lr,trainer):
    for epochs in range(num_epochs):
        for X,y in train_iter:
            with autograd.record():
                y_hat = net(X)
                l = loss(y_hat,y).sum()
            l.backward()
            
            if trainer is None:
                d2lzh.sgd(params,lr,batch_size)
            else:
                trainer.step(batch_size)
                
            y = y.asstype('float32')
            train_l_sum += l.asscalar()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().asscalar()
            n+=y.size
        test_acc = evaluate_accuracy(test_iter,net)
        

In [31]:
def train(net,train_iter,test_iter,num_epochs,batch_size,loss,trainer,params=None,lr,weight_decay=None):
    acc_sum,l_sum,n = 0.0,0.0,0
    for epoch in range(num_epochs):
        for X,y in train_iter:
            with autograd.record():
                y_hat = net(X)
                l = loss(y_hat,y)
            l.backward()
            
            if trainer is None:
                d2l.sgd(params,lr,batch_size)
            else:
                trainer.step(batch_size)
                
            l_sum += l.asscalar()
            acc_sum += (y_hat.argmax(axis=1) == y.astype('float32')).sum.asscalar()
            n+=y.size
        test_acc = evaluate_accuracy(test_iter,net)

SyntaxError: non-default argument follows default argument (<ipython-input-31-e08b1c27c157>, line 1)

In [ ]:
for X,y in test_iter:
    break
true_labels = d2l.get_fashion_mnist_labels(y.asnumpy())
pred_labels = d2l.get_fashion_mnist_labels(net(X).argmax(axis=1).asnumpy())
title = [true + '\n' + pred for true,pred in zip(true_labels,pred_labels)]

